In [1]:
# import torch_geometric
import sys
sys.path.insert(0, '../src')

In [2]:
import torch.nn.functional as F
import torchvision.transforms as transforms
import torch.optim as optim
import torchvision
import json
import argparse
# import cv2
import numpy as np
import torch
from torch.autograd import Function
from torchvision import models
import torch.nn as nn
import os
import matplotlib.pyplot as plt
import time
from torchvision.datasets import CocoDetection


from PIL import Image
import pandas as pd

sys.path.insert(0, '../src/data/cocoapi/PythonAPI')
from pycocotools.coco import COCO

In [3]:
# https://github.com/jlevy44/WSI-GTFE/blob/master/notebooks/3_fit_gnn_model.ipynb
import torch, torch.nn as nn
from torch_geometric.nn import GCNConv, GATConv, DeepGraphInfomax, SAGEConv
from torch_geometric.nn import DenseGraphConv
from torch_geometric.utils import to_dense_batch, to_dense_adj, dense_to_sparse
from torch_geometric.nn import GINEConv
from torch_geometric.utils import dropout_adj


class GCNNet(torch.nn.Module):
    def __init__(self, inp_dim, out_dim, hidden_topology=[32,64,128,128], p=0.5, p2=0.1, drop_each=True):
        super(GCNNet, self).__init__()
        self.out_dim=out_dim
        self.convs = nn.ModuleList([GATConv(inp_dim, hidden_topology[0])]+[GATConv(hidden_topology[i],hidden_topology[i+1]) for i in range(len(hidden_topology[:-1]))])
        self.drop_edge = lambda edge_index: dropout_adj(edge_index,p=p2)[0]
        self.dropout = nn.Dropout(p)
        self.fc = nn.Linear(hidden_topology[-1], out_dim)
        self.drop_each=drop_each

    def forward(self, x, edge_index, edge_attr=None):
        for conv in self.convs:
            if self.drop_each and self.training: edge_index=self.drop_edge(edge_index)
            x = F.relu(conv(x, edge_index, edge_attr))
        if self.training:
            x = self.dropout(x)
        x = self.fc(x)
        return x

In [4]:
model_gcn=GCNNet(2048,80, [32]*2)

In [5]:
model_mlp=nn.Sequential(nn.Sequential(nn.Linear(2048,32),nn.ReLU(),nn.Dropout(p=0.3)),nn.Linear(32,80))

In [6]:
model_gcn.cuda()

GCNNet(
  (convs): ModuleList(
    (0): GATConv(2048, 32, heads=1)
    (1): GATConv(32, 32, heads=1)
  )
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=32, out_features=80, bias=True)
)

In [7]:
model_mlp

Sequential(
  (0): Sequential(
    (0): Linear(in_features=2048, out_features=32, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.3, inplace=False)
  )
  (1): Linear(in_features=32, out_features=80, bias=True)
)

In [8]:
import PIL

In [9]:
# ! jupyter nbextension enable --py widgetsnbextension
# ! python -c "from torchvision import models; model = models.resnet50(pretrained=True)" 


In [10]:
model = models.resnet50(pretrained=False)


In [11]:
pwd

'/home/jdlevy/COGS_185/cogs_185_final_project/notebooks'

In [12]:
os.getcwd()

'/home/jdlevy/COGS_185/cogs_185_final_project/notebooks'

In [13]:
state_dict=torch.load("../src/models/resnet50-19c8e357.pth")
model.load_state_dict(state_dict)
# model.fc = nn.Linear(2048, 80)

<All keys matched successfully>

In [14]:
model.fc=nn.Flatten()

In [15]:
def crop_by_bbox(img):
#     print(img)
#     print('hi')
    return img

In [16]:
# ! git clone https://github.com/cocodataset/cocoapi.git

In [17]:
import sys

In [18]:
from PIL import Image

In [19]:
# class cocoDataset(torch.utils.data.Dataset):
#     def __init__(self, root, annotation, transforms=None):
#         self.root = root
#         self.transforms = transforms
#         self.coco = COCO(annotation)

In [20]:
# Source: https://medium.com/fullstackai/how-to-train-an-object-detector-with-your-own-coco-dataset-in-pytorch-319e7090da5
class cocoDataset(torch.utils.data.Dataset):
    def __init__(self, root, annotation, transforms=None, target_transform=None):
        self.root = root
        self.transforms = transforms
        self.coco = COCO(annotation)
        self.ids = list(sorted(self.coco.imgs.keys()))
        self.target_transform = target_transform

    
    def __getitem__(self, index):
        # Coco File
        coco = self.coco
        # Image ID
        img_id = self.ids[index]
        # List: get annotation id for img from coco
        ann_ids = coco.getAnnIds(imgIds=img_id)
        # Dictionary: coco annotations for an image
        coco_annotation = coco.loadAnns(ann_ids)
        # path for input image
        path = coco.loadImgs(img_id)[0]['file_name']
        # open the input image with PIL
        img = Image.open(os.path.join(self.root, path)).convert("RGB")
        
        # number of objects in the image
        num_objs = len(coco_annotation)
        
        # Extracting bounding boxes for objects
        # In coco format, bbox = [xmin, ymin, xmax, ymax]
        # In pytorch, input should be [xmin, ymin, xmax, ymax]
        boxes = []
        # Labels for each objet
        labels = []
        # Area of each bounding box
        areas = []
        # IsCrowd: whether or not the object is a crowd of objects
        iscrowd = []
        xy=[]
        subimages=[]
        for i in range(num_objs):
            xmin = coco_annotation[i]['bbox'][0]
            ymin = coco_annotation[i]['bbox'][1]
            xmax = xmin + coco_annotation[i]['bbox'][2]
            ymax = ymin + coco_annotation[i]['bbox'][3]
            bbox=np.array([xmin, ymin, xmax, ymax]).astype(int)
            boxes.append(bbox)
            labels.append(coco_annotation[i]['category_id'])
            areas.append(coco_annotation[i]['area'])
            iscrowd.append(coco_annotation[i]['iscrowd'])
            xy.append([(xmax-xmin)/2,(ymax-ymin)/2])
            subimages.append(img.crop(bbox))
            
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        areas = torch.as_tensor(areas, dtype=torch.float32)
        iscrowd = torch.as_tensor(iscrowd, dtype=torch.float32)
        
        # Annotation is in dictionary format
        my_annotation = {}
        my_annotation['boxes'] = boxes
        my_annotation['labels'] = labels
        my_annotation['image_id'] = img_id
        my_annotation['area'] = areas
        my_annotation['iscrowd'] = iscrowd
        
        imgs=[img]
        
        if self.transforms is not None:
            imgs = [self.transforms(img) for img in subimages]
        if self.target_transform is not None:
            labels = [self.target_transform(label) for label in labels]
#         if self.
        labels = torch.as_tensor(labels, dtype=torch.int64)
            
        imgs=torch.stack(imgs)
        
        xy=torch.tensor(xy)
            
        return imgs, xy, torch.LongTensor(labels)
    
    def __len__(self):
        return len(self.ids)
        

In [21]:
### TODO: add target transform to convert labels to proper labels using id_dict

In [22]:
def load_data(annFile):
    # import json
    with open(annFile, 'r') as f:
        data = json.load(f)
    coco = COCO(annFile)
    # _, data['categories'] = fix_ids(data)
    return coco, data

def fix_ids(data):
    '''
    Takes in 'categories' key of a COCO dataset, returns new IDs for those categories (properly mapped from 0 to len(categories)-1)
    '''
    categories = data['categories']
    id_dict = {}
    for idx, cat in enumerate(categories):
        id_dict[cat['id']] = idx
        cat['id'] = idx
    return id_dict, categories

def drop_null_annotations(coco, data, dataDir, dataType, annFile, overwrite, map_ids=True, save=True, tmpDataDir='data/temp'):
    """
    Takes in a json.load(f) of an annotation file, finds all image ids without an annotation, then drops those images from the file.
    Writes to a new json file without those images.
    """
    # Writes new data to a cleaned json instances file
    fname = tmpDataDir+f"/annotations/clean_instances_{dataType}.json"
    if os.path.isfile(fname) and overwrite==False: print(fname, 'already exists')
    elif os.path.isfile(fname) and overwrite==True: print(f'{fname} already exists, overwriting anyways b/c overwrite=True')
        # if map_ids:
        #     new_data = data.copy()
        #     id_dict = {}
        #     for i, cat in enumerate(new_data['categories']):
        #         id_dict[cat['id']] = i
        #         cat['id'] = i

        # return id_dict


    images_pd = pd.Series(data['images'])
    new_images_pd = images_pd.copy()
    new_data = data.copy()
    new_data['images'] = \
        new_images_pd.loc[~images_pd.apply(lambda x: len(coco.getAnnIds(x['id']))==0)].tolist()
    # sets new_data['images'] to only the list of images with one or more annotations
#     if os.path.isdir(tmpDataF+"/annotations") == False: os.mkdir(dataDir+"/annotations")
    if map_ids: id_dict, _ = fix_ids(new_data)
    print(os.getcwd())

    # print(fname)
    if save:
        try:
            open(fname, 'w')
        except FileNotFoundError:
            os.mkdir(tmpDataDir+'/annotations')
            print('annotation directory created')
        with open(fname, 'w') as f:
            json.dump(new_data, f)
    else:
        print("file saving skipped")
    print("Start images:", len(data['images']))
    print("Images remaining:",len(new_data['images']))
    print("Number of images with no annotations:",len(data['images'])-len(new_data['images']))
    if map_ids: return id_dict
    return new_data

In [23]:

def get_transform():
    custom_transforms = []
    custom_transforms.append(torchvision.transforms.Resize(size=(128, 128)))
    custom_transforms.append(torchvision.transforms.ToTensor())
    return torchvision.transforms.Compose(custom_transforms)

In [24]:
# os.listdir('/datasets/COCO-2017/')

In [25]:
train_dir = '/datasets/COCO-2017/train2017'
val_dir = '/datasets/COCO-2017/val2017'

train_ann = '/datasets/COCO-2017/anno2017/instances_train2017.json'
val_ann = '/datasets/COCO-2017/anno2017/instances_val2017.json'

In [26]:
val_coco, val_data = load_data(val_ann)


loading annotations into memory...
Done (t=1.87s)
creating index...
index created!


In [27]:
pwd

'/home/jdlevy/COGS_185/cogs_185_final_project/notebooks'

In [28]:
id_dict = drop_null_annotations(coco=val_coco, data=val_data, dataDir='/datasets/COCO-2017', dataType='val2017',
                      annFile=val_ann, overwrite=True, tmpDataDir='../data/temp')

../data/temp/annotations/clean_instances_val2017.json already exists, overwriting anyways b/c overwrite=True
/home/jdlevy/COGS_185/cogs_185_final_project/notebooks
Start images: 5000
Images remaining: 4952
Number of images with no annotations: 48


In [29]:
def convert_ids(target):
    return id_dict[target]

In [30]:
# # %%capture
# train_coco, train_data = load_data(train_ann)
# %time drop_null_annotations(coco=train_coco, data=train_data, dataDir='/datasets/COCO-2017', dataType='train2017',\
#                             annFile=train_ann, overwrite=True, tmpDataDir='../data/temp');

In [31]:
# del train_coco
# del train_data

# del val_coco
# del val_data

In [32]:
clean_train_ann = '../data/temp/annotations/clean_instances_train2017.json'
clean_val_ann = '../data/temp/annotations/clean_instances_val2017.json'

In [33]:
%time train_set = cocoDataset(root=train_dir,\
                      annotation=clean_train_ann,\
                      transforms=get_transform(),\
                      target_transform=convert_ids)

%time val_set = cocoDataset(root=val_dir,\
                      annotation=clean_val_ann,\
                      transforms=get_transform(),\
                      target_transform=convert_ids)

loading annotations into memory...
Done (t=57.03s)
creating index...
index created!
CPU times: user 32.2 s, sys: 8.94 s, total: 41.2 s
Wall time: 1min 7s
loading annotations into memory...
Done (t=1.96s)
creating index...
index created!
CPU times: user 1 s, sys: 248 ms, total: 1.25 s
Wall time: 2.05 s


In [40]:
def collate_fn(batch):
    imgs=torch.cat([item[0] for item in batch],dim=0)
    xy=torch.cat([item[1] for item in batch],dim=0)
    y=torch.cat([item[2] for item in batch]).flatten()
    return [imgs,xy,y]

In [46]:
## Params cell
num_epoch = 20
fname = '80_20_resnet50.pth'
train_batch_size = 12

In [47]:


trainloader = torch.utils.data.DataLoader(train_set,
                                          batch_size=train_batch_size,
                                          shuffle=True,
                                          num_workers=8,
                                          collate_fn=collate_fn)

valloader = torch.utils.data.DataLoader(val_set,
                                        batch_size=train_batch_size,
                                        shuffle=True,
                                        num_workers=8,
                                        collate_fn=collate_fn)



In [48]:
# If there are GPUs, choose the first one for computing. Otherwise use CPU.
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
# If 'cuda:0' is printed, it means GPU is available.

model = models.resnet50(pretrained=False)
state_dict=torch.load("../src/models/resnet50-19c8e357.pth")
model.load_state_dict(state_dict)
model.fc = nn.Linear(2048, 80)
model.to(device)
print(f'model parameters sent to {device}')

loss_func = nn.CrossEntropyLoss(reduction='mean')
opt = optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)

avg_losses = [] # Average losses
epochs = num_epoch
print_freq = 100

for epoch in range(epochs):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        #print(data)
        # Get the inputs.
        inputs, _, labels = data
        
        try:
            # Move the inputs to the specified device
            inputs, labels = inputs.to(device), labels.to(device)

            # Zero the parameter gradients.
            opt.zero_grad()

            # Forward step.
            outputs = model(inputs)
            loss = loss_func(outputs, labels)

            # Backward step.
            loss.backward()

            # Optimization step (update the parameters).
            opt.step()

            # Print statistics.
            running_loss += loss.item()
#         except RuntimeError:
#             print(f'Input shape: {inputs.shape}\n Label shape: {labels.shape}')
#             print(f' Labels: {labels}')
            
        if i % print_freq == print_freq - 1: # prints every several mini-batches.
            avg_loss = running_loss / print_freq
            print(f'[epoch: {epoch}, i: {i}] avg mini-batch loss: {avg_loss}')
            avg_losses.append(avg_loss)
            running_loss = 0.0
        
        del inputs, outputs, labels, loss
print('Finished Training.')

SyntaxError: invalid syntax (<ipython-input-48-12dacc98e232>, line 50)

In [107]:
! ls /datasets/COCO-2017/val2017/ | grep 000000311913

In [44]:
# plot
len(avg_losses)

5670

In [45]:
# '/datasets/COCO-2017/val2017'

In [45]:
PATH = f'../src/models/{fname}'
torch.save(model.state_dict(), PATH)
print(f'Model saved at {PATH}')

Model saved at ../src/models/80_20_resnet50.pth


In [48]:
model.cuda()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [46]:
from torch_geometric.data import Data
from torch_cluster import knn_graph
graphs=[]
for i,(imgs,xy,y) in enumerate(valloader):
    print(imgs.shape,xy.shape,y.shape)
    edge_index = knn_graph(xy,k=5)
    graphs.append(Data(x=model(imgs),pos=xy,edge_index=edge_index,y=y))
#     print(graphs[i].x.shape)
    if i==10: break

torch.Size([1, 3, 128, 128]) torch.Size([1, 2]) torch.Size([1])


RuntimeError: Input type (torch.FloatTensor) and weight type (torch.cuda.FloatTensor) should be the same

In [83]:
from torch_geometric.data import DataLoader as TG_DataLoader
# https://pytorch.org/docs/stable/data.html#torch.utils.data.Subset

graph_dataloader=TG_DataLoader(graphs,batch_size=2,shuffle=True)#[0]

In [87]:
for G in graph_dataloader:
    y=G.y
    z=G.x
    edge_index=G.edge_index
    y_pred=model_gcn(z,edge_index)
    print(G)

NameError: name 'F' is not defined

In [89]:
# select device (whether GPU or CPU)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# DataLoader is iterable over Dataset
for imgs, annotations in valloader:
    imgs = list(img.to(device) for img in imgs)
    annotations = [{k: v.to(device) for k, v in t.items()} for t in annotations]
    print(annotations)

ValueError: too many values to unpack (expected 2)

In [17]:
# train_set

Dataset CocoDetection
    Number of datapoints: 118287
    Root location: /datasets/COCO-2017/train2017
    StandardTransform
Transform: <function crop_by_bbox at 0x7fe92a5d53b0>

In [20]:
val_set

Dataset CocoDetection
    Number of datapoints: 5000
    Root location: /datasets/COCO-2017/val2017
    StandardTransform
Transform: <function crop_by_bbox at 0x7fe92a5d53b0>

In [51]:
model_gcn=GCNNet(2048,80, [32]*2)

In [52]:
state_dict=torch.load("../src/models/graph_80_5_resnet50.pth")
model_gcn.load_state_dict(state_dict)
# model.fc = nn.Linear(2048, 80)

<All keys matched successfully>